In [1]:
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

In [2]:
#data_path = '/Users/chiaracavigliano/Desktop/Neuroengineering/Autism_project/Matlab_code/Data'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/Progetto_Neuro'

In [4]:
## SPLIT
count = 0
for patient in os.listdir(data_path):
    if patient.startswith('.') or patient in ['bin', 'include', 'lib', '.venv', 'pyvenv.cfg', '.gitignore'] or patient.endswith('.csv'):
        continue
    count = count+1
print(count)

## farei 80/10/10 --> da aggiustare a seconda del numero totale di pazienti
train_p = 25
val_p = 6
test_p = 0

print(f'Training patients: ',train_p)
print(f'Validation patients: ', val_p)
print(f'Test patients: ', test_p)
print(f'Total number of patients: ', count)

39
Training patients:  25
Validation patients:  6
Test patients:  0
Total number of patients:  39


In [5]:
import os
import pandas as pd

dataset = {}

# Itera attraverso le cartelle nella directory
for patient in os.listdir(data_path):
    if patient.startswith('.') or patient in ['bin', 'include', 'lib', '.venv', 'pyvenv.cfg', '.gitignore']:
        continue

    patient_path = os.path.join(data_path, patient)

    if os.path.isdir(patient_path):  # Verifica se è una cartella
        selected_gaze_path = os.path.join(patient_path, 'selected_gaze_th.txt')

        if os.path.isfile(selected_gaze_path):  # Controlla se il file 'selected_gaze_ch.txt' esiste nella cartella
            try:
                # Prova a caricare il file in un DataFrame
                df = pd.read_csv(selected_gaze_path, sep='\s+', header=None)
                dataset[patient] = df  # Salva il DataFrame sotto il nome della cartella
                print(f"Caricato il terapista {patient}")
            except Exception as e:
                print(f"Errore nel caricamento di {selected_gaze_path}: {e}")

print(f"Dataset completato per {len(dataset)} therapist.")


Caricato il terapista Lazarevski_Cavigliano_robotRIFARE
Errore nel caricamento di /content/drive/MyDrive/Progetto_Neuro/Lazarevski_Cavigliano_norobotRIFARE/selected_gaze_th.txt: No columns to parse from file
Caricato il terapista Zeppa_Macchini_robotOK
Caricato il terapista Zeppa_Macchini_norobotOK
Caricato il terapista Zannino_Martella_robotOK
Caricato il terapista Simeone_Dalfonso_robotOK
Caricato il terapista Simeone_Dalfonso_norobotOK
Caricato il terapista Prizzi_Mura_robotOK
Caricato il terapista Prizzi_Mura_norobotOK
Caricato il terapista Mura_Prizzi_robotOK
Caricato il terapista Mura_Prizzi_norobotOK
Caricato il terapista Cavigliano_Lazarevski_robotOK
Caricato il terapista Cavigliano_Lazarevski_norobotOK
Caricato il terapista Dalfonso_Simeone_robotOK
Caricato il terapista Dalfonso_Simeone_norobotOK
Caricato il terapista Magnani_Faures_robotOK
Caricato il terapista Magnani_Faures_norobotOK
Caricato il terapista Macchini_Zeppa_robotOK
Caricato il terapista Macchini_Zeppa_norobotOK

In [6]:
dataset.keys()

dict_keys(['Lazarevski_Cavigliano_robotRIFARE', 'Zeppa_Macchini_robotOK', 'Zeppa_Macchini_norobotOK', 'Zannino_Martella_robotOK', 'Simeone_Dalfonso_robotOK', 'Simeone_Dalfonso_norobotOK', 'Prizzi_Mura_robotOK', 'Prizzi_Mura_norobotOK', 'Mura_Prizzi_robotOK', 'Mura_Prizzi_norobotOK', 'Cavigliano_Lazarevski_robotOK', 'Cavigliano_Lazarevski_norobotOK', 'Dalfonso_Simeone_robotOK', 'Dalfonso_Simeone_norobotOK', 'Magnani_Faures_robotOK', 'Magnani_Faures_norobotOK', 'Macchini_Zeppa_robotOK', 'Macchini_Zeppa_norobotOK', 'Martella_Zannino_robotOK', 'Martella_Zannino_norobotOK', 'Faures_Magnani_robotOK', 'Faures_Magnani_norobotOK', 'Dussin_Bosetti_robotOK', 'Dussin_Bosetti_norobotOK', 'DeGiorgis_Zanzottera_norobotOK', 'Bosetti_Dussin_norobotOK', 'Anchisi_Sonnacchini_norobotOK'])

In [7]:
dataset['Zeppa_Macchini_robotOK'].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,5,0.099493,0.099493,0.605377,-0.532072,-0.825140,-0.148584,2.574686,617,414,656,476,637.006653,436.387760,R,nowhere,1.732537e+09,ZeMa_TH
1,10,0.084374,0.084374,0.523105,-0.521266,-0.825666,-0.150828,2.592806,616,414,656,476,636.854126,436.574672,R,nowhere,1.732537e+09,ZeMa_TH
2,15,0.068990,0.068990,0.360742,-0.462327,-0.828398,-0.150527,2.597599,617,414,657,476,637.396240,436.692218,R,nowhere,1.732537e+09,ZeMa_TH
3,20,0.072228,0.072228,0.389051,-0.505239,-0.831130,-0.150227,2.602393,617,414,657,476,637.316101,436.788017,R,nowhere,1.732537e+09,ZeMa_TH
4,25,0.084819,0.084819,0.532039,-0.492765,-0.833862,-0.149926,2.607187,617,414,657,476,637.221680,436.727763,R,nowhere,1.732537e+09,ZeMa_TH


Shuffle

In [ ]:
#### DA SISTEMARE ####
dataset = pd.DataFrame.from_dict(dataset, orient='index')
dataset.head()

indices = np.arange(dataset.shape[0])
np.random.shuffle(indices)
print(indices)
dataset = dataset.iloc[indices, :]
dataset.head()

[0 6 1 7 2 3 5 4]


,labels_ch,selected_gaze_ch
p1,0 0 0 1 -1 2 -1 3 -1 4 -1 .. ...,0 1 2 3 4...
p5,0 0 2 1 -1 2 -1 3 -1 4 -1 .. ...,0 1 2 3 4...
p2,0 0 0 1 -1 2 -1 3 -1 4 -...,0 1 2 3 ...
p8,0 0 1 1 -1 2 -1 3 -1 4 -1 .. ...,0 1 2 3 ...
p4,0 0 0 1 -1 2 -1 3 -1 4 -...,0 1 2 3 ...


In [8]:
dataset.keys()

dict_keys(['Lazarevski_Cavigliano_robotRIFARE', 'Zeppa_Macchini_robotOK', 'Zeppa_Macchini_norobotOK', 'Zannino_Martella_robotOK', 'Simeone_Dalfonso_robotOK', 'Simeone_Dalfonso_norobotOK', 'Prizzi_Mura_robotOK', 'Prizzi_Mura_norobotOK', 'Mura_Prizzi_robotOK', 'Mura_Prizzi_norobotOK', 'Cavigliano_Lazarevski_robotOK', 'Cavigliano_Lazarevski_norobotOK', 'Dalfonso_Simeone_robotOK', 'Dalfonso_Simeone_norobotOK', 'Magnani_Faures_robotOK', 'Magnani_Faures_norobotOK', 'Macchini_Zeppa_robotOK', 'Macchini_Zeppa_norobotOK', 'Martella_Zannino_robotOK', 'Martella_Zannino_norobotOK', 'Faures_Magnani_robotOK', 'Faures_Magnani_norobotOK', 'Dussin_Bosetti_robotOK', 'Dussin_Bosetti_norobotOK', 'DeGiorgis_Zanzottera_norobotOK', 'Bosetti_Dussin_norobotOK', 'Anchisi_Sonnacchini_norobotOK'])

In [9]:
for therapist in dataset:
    print(therapist)

Lazarevski_Cavigliano_robotRIFARE
Zeppa_Macchini_robotOK
Zeppa_Macchini_norobotOK
Zannino_Martella_robotOK
Simeone_Dalfonso_robotOK
Simeone_Dalfonso_norobotOK
Prizzi_Mura_robotOK
Prizzi_Mura_norobotOK
Mura_Prizzi_robotOK
Mura_Prizzi_norobotOK
Cavigliano_Lazarevski_robotOK
Cavigliano_Lazarevski_norobotOK
Dalfonso_Simeone_robotOK
Dalfonso_Simeone_norobotOK
Magnani_Faures_robotOK
Magnani_Faures_norobotOK
Macchini_Zeppa_robotOK
Macchini_Zeppa_norobotOK
Martella_Zannino_robotOK
Martella_Zannino_norobotOK
Faures_Magnani_robotOK
Faures_Magnani_norobotOK
Dussin_Bosetti_robotOK
Dussin_Bosetti_norobotOK
DeGiorgis_Zanzottera_norobotOK
Bosetti_Dussin_norobotOK
Anchisi_Sonnacchini_norobotOK


In [10]:
import pandas as pd

# Lista per raccogliere i DataFrame dei pazienti
all_selected_gaze = []

# Itera sui pazienti nel dataset fino a quelli indicati da train_p
for counter, patient in enumerate(list(dataset.keys())[:train_p]):  # Itera sui primi train_p pazienti
    print(f"Processando i dati per il paziente: {patient}")

    patient_data = dataset[patient]  # Ottieni i dati per il paziente corrente

    # Aggiungi il DataFrame del paziente alla lista
    all_selected_gaze.append(patient_data)

# Verifica se i dati sono stati raccolti prima di creare il DataFrame
if all_selected_gaze:
    final_selected_gaze = pd.concat(all_selected_gaze, ignore_index=True)

    # Salva i dati nel file CSV
    output_gaze = os.path.join(data_path, "Book_train_th.csv")
    final_selected_gaze.to_csv(output_gaze, index=False, header=False)
    print(f"File CSV creato per i dati di gaze: {output_gaze}")
else:
    print("Nessun dato da scrivere nel CSV.")


Processando i dati per il paziente: Lazarevski_Cavigliano_robotRIFARE
Processando i dati per il paziente: Zeppa_Macchini_robotOK
Processando i dati per il paziente: Zeppa_Macchini_norobotOK
Processando i dati per il paziente: Zannino_Martella_robotOK
Processando i dati per il paziente: Simeone_Dalfonso_robotOK
Processando i dati per il paziente: Simeone_Dalfonso_norobotOK
Processando i dati per il paziente: Prizzi_Mura_robotOK
Processando i dati per il paziente: Prizzi_Mura_norobotOK
Processando i dati per il paziente: Mura_Prizzi_robotOK
Processando i dati per il paziente: Mura_Prizzi_norobotOK
Processando i dati per il paziente: Cavigliano_Lazarevski_robotOK
Processando i dati per il paziente: Cavigliano_Lazarevski_norobotOK
Processando i dati per il paziente: Dalfonso_Simeone_robotOK
Processando i dati per il paziente: Dalfonso_Simeone_norobotOK
Processando i dati per il paziente: Magnani_Faures_robotOK
Processando i dati per il paziente: Magnani_Faures_norobotOK
Processando i dati 

In [11]:
import pandas as pd

# Lista per raccogliere i DataFrame dei pazienti
all_selected_gaze = []
counter=0

# Itera sui pazienti nel dataset fino a quelli indicati da train_p
for counter, patient in enumerate(list(dataset.keys())[train_p:train_p+val_p]):
    print(f"Processando i dati per il paziente: {patient}")

    patient_data = dataset[patient]  # Ottieni i dati per il paziente corrente

    # Aggiungi il DataFrame del paziente alla lista
    all_selected_gaze.append(patient_data)

# Verifica se i dati sono stati raccolti prima di creare il DataFrame
if all_selected_gaze:
    final_selected_gaze = pd.concat(all_selected_gaze, ignore_index=True)

    # Salva i dati nel file CSV
    output_gaze = os.path.join(data_path, "Book_val_th.csv")
    final_selected_gaze.to_csv(output_gaze, index=False, header=False)
    print(f"File CSV creato per i dati di gaze: {output_gaze}")
else:
    print("Nessun dato da scrivere nel CSV.")


Processando i dati per il paziente: Bosetti_Dussin_norobotOK
Processando i dati per il paziente: Anchisi_Sonnacchini_norobotOK
File CSV creato per i dati di gaze: /content/drive/MyDrive/Progetto_Neuro/Book_val_th.csv


In [ ]:
## preparo Book e labels per il training
all_selected_gaze = []
all_selected_labels = []
counter = 0


for patient in dataset:
    counter = counter + 1
    print(patient)
    for file in dataset[patient]:
        if file == 'selected_gaze_ch':
            all_selected_gaze.append(dataset[patient]["selected_gaze_ch"])
        if file == 'labels_ch':
            all_selected_labels.append(dataset[patient]["labels_ch"])
    if counter == train_p:
        break

# Concatena tutti i DataFrame in uno unico
final_selected_gaze = pd.concat(all_selected_gaze, ignore_index=True)
final_selected_labels = pd.concat(all_selected_labels, ignore_index=True)


output_gaze = os.path.join(data_path, "Book_train.csv")
final_selected_gaze.to_csv(output_gaze, index=False, header=False)
#output_labels = os.path.join(data_path, "labels_train.csv")
#final_selected_labels.to_csv(output_labels, index=False, header=False)

print(f"File CSV creato: {output_gaze}")
#print(f"File CSV creato: {output_labels}")

Zeppa_Macchini_robotOK
Zeppa_Macchini_norobotOK
Zanzottera_Degiorgis_robotOK
Zannino_Martella_robotOK
Simeone_Dalfonso_robotOK
Simeone_Dalfonso_norobotOK
Prizzi_Mura_robotOK
Prizzi_Mura_norobotOK
Mura_Prizzi_robotOK
Mura_Prizzi_norobotOK
Cavigliano_Lazarevski_robotOK
Cavigliano_Lazarevski_norobotOK
Dalfonso_Simeone_robotOK
Dalfonso_Simeone_norobotOK
Magnani_Faures_robotOK
Magnani_Faures_norobotOK
Macchini_Zeppa_robotOK
Macchini_Zeppa_norobotOK
Martella_Zannino_robotOK
Martella_Zannino_norobotOK
Faures_Magnani_robotOK
Faures_Magnani_norobotOK
Dussin_Bosetti_robotOK
Dussin_Bosetti_norobotOK


ValueError: No objects to concatenate

In [ ]:
## Book e labels per il validation
all_selected_gaze = []
all_selected_labels = []
counter = 0


for patient in dataset:
    #print(counter)
    counter = counter + 1
    if counter <= train_p:
        continue
    print(patient)
    for file in dataset[patient]:
        if file == 'selected_gaze_ch':
            all_selected_gaze.append(dataset[patient]["selected_gaze_ch"])
        if file == 'labels_ch':
            all_selected_labels.append(dataset[patient]["labels_ch"])
    if counter == val_p + train_p:
        break

# Concatena tutti i DataFrame in uno unico
final_selected_gaze = pd.concat(all_selected_gaze, ignore_index=True)
final_selected_labels = pd.concat(all_selected_labels, ignore_index=True)


output_gaze = os.path.join(data_path, "Book_val.csv")
final_selected_gaze.to_csv(output_gaze, index=False, header=False)
#output_labels = os.path.join(data_path, "labels_val.csv")
#final_selected_labels.to_csv(output_labels, index=False, header=False)

print(f"File CSV creato: {output_gaze}")
#print(f"File CSV creato: {output_labels}")

DeGiorgis_Zanzottera_robotOK
DeGiorgis_Zanzottera_norobotOK
Bosetti_Dussin_robotOK
Bosetti_Dussin_norobotOK
Anchisi_Sonnacchini_robotOK
Anchisi_Sonnacchini_norobotOK


ValueError: No objects to concatenate

In [ ]:
## Book e labels per il test
all_selected_gaze = []
all_selected_labels = []
counter = 0


for patient in dataset:
    #print(counter)
    counter = counter + 1
    if counter <= train_p + val_p:
        print(counter)
        continue
    print(patient)
    for file in dataset[patient]:
        if file == 'selected_gaze_ch':
            all_selected_gaze.append(dataset[patient]["selected_gaze_ch"])
        if file == 'labels_ch':
            all_selected_labels.append(dataset[patient]["labels_ch"])

    if counter == val_p + train_p + test_p:
        print('break')
        break

# Concatena tutti i DataFrame in uno unico
final_selected_gaze = pd.concat(all_selected_gaze, ignore_index=True)
final_selected_labels = pd.concat(all_selected_labels, ignore_index=True)


output_gaze = os.path.join(data_path, "Book_test.csv")
final_selected_gaze.to_csv(output_gaze, index=False, header=False)
output_labels = os.path.join(data_path, "labels_test.csv")
final_selected_labels.to_csv(output_labels, index=False, header=False)

print(f"File CSV creato: {output_gaze}")
print(f"File CSV creato: {output_labels}")

1
2
3
4
5
6
7
p8
break
File CSV creato: /content/drive/MyDrive/Progetto_Neuro/Modello/Data/Book_test.csv
File CSV creato: /content/drive/MyDrive/Progetto_Neuro/Modello/Data/labels_test.csv


In [ ]:
with open('D:\\Pickles_attention\\no_hbb_me_train.pkl', 'wb') as f:
  pickle.dump(data, f)

In [ ]:
# Carica il file CSV in un DataFrame
csv_file_path = os.path.join(data_path, 'Book_train.csv')
df = pd.read_csv(csv_file_path)

pickle_directory = '/content/drive/MyDrive/Progetto_Neuro/Modello/pickles'
pickle_file_path = os.path.join(pickle_directory, 'no_hbb_me_train.pkl')
df.to_pickle(pickle_file_path)

In [ ]:
# Carica il file CSV in un DataFrame
csv_file_path = os.path.join(data_path, 'Book_val.csv')
df = pd.read_csv(csv_file_path)

pickle_directory = '/content/drive/MyDrive/Progetto_Neuro/Modello/pickles'
pickle_file_path = os.path.join(pickle_directory, 'no_hbb_me_val.pkl')
df.to_pickle(pickle_file_path)

In [ ]:
# Carica il file CSV in un DataFrame
csv_file_path = os.path.join(data_path, 'Book_test.csv')
df = pd.read_csv(csv_file_path)

pickle_directory = '/content/drive/MyDrive/Progetto_Neuro/Modello/pickles'
pickle_file_path = os.path.join(pickle_directory, 'no_hbb_me_test.pkl')
df.to_pickle(pickle_file_path)

In [ ]:
# Carica il file CSV in un DataFrame
csv_file_path = os.path.join(data_path, 'Book_test.csv')
df = pd.read_csv(csv_file_path)

pickle_directory = '/content/drive/MyDrive/Progetto_Neuro/Modello/pickles'
pickle_file_path = os.path.join(pickle_directory, 'no_hbb_me_test.pkl')
df.to_pickle(pickle_file_path)